In [ ]:
import pandas as pd
import numpy as np
import gc

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv( "test.csv")
labels = pd.read_csv( "test_labels.csv")

In [ ]:
train = train[:100000]
classes = list(train.columns)[2:]
classes

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [ ]:
links = '(http://.*?\s)|(http://.*)'
ip_addr = '\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}'
users = '\[\[User.*'
newline = '\\n'
def clean(comment):
  import re
  comment=comment.lower()
  comment=re.sub(links,'',comment)
  comment=re.sub(ip_addr,'',comment)
  comment=re.sub(users,'',comment)
  comment=re.sub(newline,'',comment)
  return comment

In [ ]:
train['comment_text']=train['comment_text'].map(lambda i:clean(i))
test['comment_text']=test['comment_text'].map(lambda i:clean(i))

In [ ]:
x=train['comment_text']
y=train.iloc[:,2:8]
print(x.shape)
print(y.shape)

(100000,)
(100000, 6)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(x,y,test_size=0.2,random_state=1)

In [ ]:
labels = pd.read_csv("test_labels.csv")
labels=labels.iloc[:,1:]
sum_labels=np.sum(labels,axis=1)
idx=sum_labels>=0

In [ ]:
labels_consider=labels[idx]
test=test.iloc[:,1:]
tests_consider=test[idx].values[:,0]

print(labels_consider.shape, tests_consider.shape)

(63978, 6) (63978,)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(x,y,test_size=0.2,random_state=1)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(max_features=5000,strip_accents='unicode',stop_words='english',token_pattern=r'\w{2,}')

In [ ]:
tfidf.fit(X_train)
X_train_feat=tfidf.fit_transform(X_train)
print(X_train_feat.shape)

X_val_feat=tfidf.transform(X_val)
print(X_val_feat.shape)

X_test_feat=tfidf.transform(tests_consider)
print(X_test_feat.shape)

(80000, 5000)
(20000, 5000)
(63978, 5000)


In [ ]:
!pip install scikit-multilearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 3.8 MB/s eta 0:00:00


In [ ]:
print(X_train.shape)
print(y_train.shape)

(80000,)
(80000, 6)


In [ ]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.svm import SVC

model_br = BinaryRelevance(classifier = SVC(), require_dense = [False, True])
model_br.fit(X_train_feat, y_train)

BinaryRelevance(classifier=SVC(), require_dense=[False, True])

In [ ]:
preds_train = model_br.predict(X_train_feat)

In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score

print(roc_auc_score(y_train, preds_train.toarray()))

0.7697550915372223


In [ ]:

preds_val = model_br.predict(X_val_feat)


In [ ]:
print(roc_auc_score(y_val, preds_val.toarray()))

0.6665239237377513


In [ ]:
preds_test = model_br.predict(X_test_feat)

In [ ]:
print(roc_auc_score(labels_consider, preds_test.toarray()))

0.6636501540392014
